In [ ]:
from pathlib import Path
import os
import hjson
import json
import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import display
from loguru import logger

from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.utils import rename_signal_ids_to_var_ids
from phd_visualizations.utils.units import unit_conversion
from phd_visualizations.test_timeseries import experimental_results_plot


# auto reload modules
%load_ext autoreload

# Paths definition
src_path = Path(f'{os.getenv("HOME")}/Nextcloud/Juanmi_MED_PSA/EURECAT/')
results_path: Path = src_path / 'results'
data_path: Path = src_path / 'data'

# filename_opt_result = '20240108_optimization_results.json'
# Debería ser un .csv al que se le hayan añadido las variables faltantes desde librescada:
# - J de variadores y medidor de potencia
# - FT-DES-002_VFD

filename_process_data = '20230807_aquasol.csv'

# Resample figures using plotly_resampler
resample_figures = False

sample_rate = '60s'
sample_rate_numeric = int(sample_rate[:-1])

In [ ]:
# Load variables information
with open( Path("data") / 'variables_config.hjson') as f:
    vars_config = hjson.load(f)

# Read data from csv, the index column is the one named "time", which is not the first one
df = pd.read_csv(data_path / filename_process_data, parse_dates=True, index_col='TimeStamp')
# Rename index column to "time"
df.index.names = ['time']
# Set UTC timezone
df = df.tz_localize('UTC')

# Sample every `sample_rate` seconds to reduce the size of the dataframe
df = df.resample(sample_rate).mean()

# Rename columns from signal_id to var_id
df = rename_signal_ids_to_var_ids(df, vars_config)

display(df.head())

# Convert units to model units
df = unit_conversion(df, vars_config, input_unit_key='units_scada', output_unit_key='units_model')

# Filter out nans until first value in Tts

# start_idx = df.index.get_loc( df['Tts_c_b'].first_valid_index() )
# df = df.iloc[start_idx:]
df.dropna(inplace=True)
# logger.info(f'Removed up to row {start_idx} to filter NaN values in thermal storage state')

# If selecting only in operation
df_on = df.copy()
df_on = df_on.where(df['Tsf_out']-df['Tsf_in']>1).dropna()

display(df.head())

In [ ]:
# Data visualization configuration

solar_field_viz_conf = {
    # General plot attributes
  "title": "Solar field",
  "subtitle": "test visualization",
  "height": 1400,
  "width": 1000,
  "margin": {"l":20, "r":200, "t":100, "b":20, "pad":5},
  "vertical_spacing": 0.03,
  "xdomain": [0, 0.85],
  "arrow_xrel_pos": 60, # seconds

  # Individual plot attributes
  # First specify the plot attributes
  # In traces, specify the variables to be plotted, the definition order controls the order of the traces in the plot
  "plots": {
    "environment": {
      "title": "Environment",
      "row_height": 0.8,
      "bg_color": "steelblue",
      "ylabels_left": ["ºC"],
      "ylabels_right": ["I (W/m<sup>2</sup>)"], # As many as n_yaxis-1

      "traces_left": [
        {
          "var_id": "Tamb",
          "mode": "lines",
          "color": "plotly_green",
          "axis_arrow": True,
          "arrow_yrel_pos": 1.05,
        },
      ],

      "traces_right": [
        {
          "var_id": "I",
          "mode": "lines",
          "color": "plotly_yellow", # plotly green
          "axis_arrow": True,
          "arrow_yrel_pos": 1.1,
        },
      ]
    },
  
    "power": {
      "title": "<b>Thermal power generated</b><br>in total and per loop</b>",
      "row_height": 1,
      "bg_color": "bg_gray", # bg gray
      "ylabels_left": ["P<sub>th</sub> (kW<sub>th</sub>)"],
      "ylabels_right": [""],

      "traces_left": [
        { "var_id": "Psf", "color": "dc_green", "width": 1.5, }
      ],
      
      "traces_right": [
        { "var_id": "Psf_l*", "fill": "tonexty", }
      ]
    },
    
    "flows": {
      "title": "<b>Total flow</b><br>and per loop</b>",
      "row_height": 1,
      "bg_color": "bg_gray", # bg gray
      "ylabels_left": ["q (m<sup>3</sup>)/h"],

      "traces_left": [
        { "var_id": "Psf", "color": "dc_green", "width": 1.5, },
        { "var_id": "Psf_l*", "fill": "tonexty", }
      ],
      
    },
    
    "temperatures": {
        "title": "<b>Temperatures</b>",
        "row_height": 1,
        "bg_color": "bg_gray", # bg gray
        "ylabels_left": ["(ºC)"],
    
        "traces_left": [
          { "var_id": "Tsf_in", "color": "plotly_red", "width": 1.5, "dash": "longdash"},
          { "var_id": "Tsf_out", "color": "plotly_red", "width": 1.5, },
          { "var_id": "Tsf_l*", "width": 1.5, }
        ],
    }
  
  },
}

In [ ]:
# TODO: Update visualization to support giving fill as a trace parameter 
# TODO: Finish updating vars_config file to add all required fields for detailed solar field

In [ ]:
from iapws import IAPWS97 as w_props

def calculate_powers(row, max_power: float = 250, min_power=0):
    try:
        # Solar field
        w_p = w_props(P=0.16, T=(row["Tsf_in"]+row["Tsf_out"])/2+273.15) # MPa, K
        row["Psf"] = row["qsf"]/3600 * w_p.rho * w_p.cp * (row["Tsf_out"]-row["Tsf_in"]) # kW
        row["Psf"] = min(row["Psf"], max_power)
        row["Psf"] = max(row["Psf"], min_power)
        
        # Solar field loops
        for loop_str in ['l1', 'l2', 'l3', 'l4']:
            row[f"Psf_{loop_str}"] = row[f"qsf_{loop_str}"]/3600 * w_p.rho * w_p.cp * (row[f"Tsf_out_{loop_str}"]-row[f"Tsf_in_{loop_str}"]) # kW
            row[f"Psf_{loop_str}"] = min(row[f"Psf_{loop_str}"], max_power)
            row[f"Psf_{loop_str}"] = max(row[f"Psf_{loop_str}"], min_power)

    except Exception as e:
        logger.error(f'Error: {e}')
        row["Pts_src"] = np.nan
        row["Pts_dis"] = np.nan
        row["Psf"] = np.nan

    return row

df = df.apply(calculate_powers, axis=1)

# How many nans?
logger.debug(f'non value Pts_src: {df["Pts_src"].isna().sum()} / {len(df)}')

In [ ]:
# Visualize test

fig = experimental_results_plot(solar_field_viz_conf, df, vars_config=vars_config, )

if resample_figures:
    fig.show_dash(
        'inline', 
        config=generate_plotly_config(fig, figure_name=f'solar_field_{df.index[0].strftime("%Y%m%d")}')
    )
else:
    fig.show(
        config=generate_plotly_config(fig, figure_name=f'solar_field_{df.index[0].strftime("%Y%m%d")}')
    )

In [ ]:
# Save figure
save_figure(
    figure_name=f"solar_field_viz_{df.index[0].strftime('%Y%m%d')}", 
    figure_path=results_path,
    fig=fig, formats=('png', 'html'), 
    width=fig.layout.width, height=fig.layout.height, scale=2
)

In [ ]:
# Calibrate model
##%% Parameter fit
import numpy as np
from models_psa.calibration.parameters_fit import (
  objective_function,
  calculate_iae, 
  calculate_ise, 
  calculate_itae
)
from models_psa.solar_field import solar_field_model
# from optimparallel import minimize_parallel
from scipy.optimize import minimize

Tin  = 'Tsf_l2_in'
Tout = 'Tsf_l2_out'
qsf  = 'q_sf_l2'

# Parameters
# beta=2.5e-3 # Irradiance model parameter
# H=0.1 # Thermal losses coefficient (J/sºC)
# Lidia, te parece bien multiplicar por 4 para tener en cuenta todo el campo
nt=1#*4 # Number of parallel tubes in each collector
npar=7*5  # Number of parallel collectors in each loop
ns=2  # Number of serial connections of collectors rows
Lt=1.15*20 # Length of the collector inner tube [m]

# Inputs 
# q_sf = df['q_sf'][i].values
Tsf_in = df[Tin].values
q_sf_ref = df[qsf].values

Tamb = df['Tamb'].values
I = df['I'].values

# msrc = np.zeros(len(data)-1, dtype=float); mdis = np.zeros(len(data)-1, dtype=float)
# for idx in range(1, len(data)-1):
#     msrc[idx] = Qsrc[idx]/60*w_props(P=0.1, T=Tt_in[idx]+273.15).rho*1e-3 # rho [kg/m³] # Convertir L/min a kg/s
#     mdis[idx] = Qdis[idx]*w_props(P=0.1, T=data.Tts_h_t[idx]+273.15).rho*1e-3 # rho [kg/m³] # Convertir L/s a kg/s

# Experimental outputs
Tsf_out = df[Tout].values

# Define optimizer inputs
inputs = [Tsf_in, q_sf_ref, I, Tamb]  # Input values
outputs = Tsf_out  # Actual output values
params = (nt, npar, ns, Lt)    # Constant model parameters
params_objective_function = {'metric': 'IAE', 'recursive':False, 'n_outputs':1, 
                             'n_parameters': 2} # 'len_outputs':[N, N]

# Set initial parameter values
initial_parameters = [4e-5, 0.1]

#         beta min, beta max    Hmin,    Hmax
bounds = ((1e-6, 1),         (0, 4))

# Perform parameter calibration
optimized_parameters = minimize(
    objective_function,
    initial_parameters,
    args=(solar_field_model, inputs, outputs, params, params_objective_function),
    bounds = bounds,
    method='Nelder-Mead'
).x

op = optimized_parameters

beta  = op[0]
H  = op[1]


In [ ]:
# Visualize model validation

# Initialize result vectors
# q_sf_mod   = np.zeros(len(df), dtype=float)
Tsf_out_mod   = np.zeros(len(df), dtype=float)

# Evaluate model
for i in range(len(df)):
    Tsf_out_mod[i] =solar_field_model(
      Tsf_in[i], q_sf_ref[i], I[i], Tamb[i], 
      # Model parameters
      beta=beta, H=H, nt=nt, np=npar, ns=ns, Lt=Lt
    )
    
# Calculate performance metrics
iae  = calculate_iae(Tsf_out, Tsf_out_mod)
ise  = calculate_ise(Tsf_out, Tsf_out_mod)
itae = calculate_itae(Tsf_out, Tsf_out_mod)

df['Tsf_l2_pred'] = Tsf_out_mod

print(f"beta: {beta:.2e}, H: {H:.2f}, IAE: {iae:.2f}, ISE: {ise:.2f}, ITAE: {itae:.2f}")



In [ ]:
fig = experimental_results_plot(solar_field_viz_conf, df, df_comp=df_mod, vars_config=vars_config, )

if resample_figures:
    fig.show_dash(
        'inline', 
        config=generate_plotly_config(fig, figure_name=f'solar_field_{df.index[0].strftime("%Y%m%d")}')
    )
else:
    fig.show(
        config=generate_plotly_config(fig, figure_name=f'solar_field_{df.index[0].strftime("%Y%m%d")}')
    )

In [ ]:
# Save figure
save_figure(
    figure_name=f"solar_field_viz_{df.index[0].strftime('%Y%m%d')}", 
    figure_path=results_path,
    fig=fig, formats=('png', 'html'), 
    width=fig.layout.width, height=fig.layout.height, scale=2
)